In [1]:
#环境准备
import pandas as pd
import requests
from urllib.parse import urlencode
from bs4 import BeautifulSoup
import re
headers ={ 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/52.0.2743.116 Safari/537.36'}
print('环境准备完成')

In [2]:
#相关参数容器
data_list = []
name_list = []
buildNo_list = []
time_list = []
sort_list = []
#获取项目总表信息
def get_tablename(URL):
    response = requests.get(URL, headers=headers)
    soup = BeautifulSoup(response.text, 'lxml')

    for idx, tr in enumerate(soup.find_all('tbody')):
        tds = tr.find_all('td')
        for i in range(0,len(tds),5):
            data_list.append({
                '1项目名称': str(tds[i].contents[0])[68:-4],
                '2所属批次': tds[i+1].contents[0],
                '3公示时间': tds[i+2].contents[0],
                '4申报楼栋': tds[i+3].contents[0],
                '5总层数': tds[i+4].contents[0]
            })
            name = str(tds[i].contents[0])[18:50]
            name_list.append(name)
            no = tds[i+3].contents[0]
            buildNo_list.append(no)
            time = tds[i+2].contents[0]
            time_list.append(time)
            sort = tds[i+1].contents[0]
            sort_list.append(sort)

In [3]:
#循环获取41页相目信息
url = 'http://117.39.29.75:8085/pricePublic/house/public/index?sort=&type=0&decprojectName=%25%25&page='
x = 1
while x < 42:
    URL = url + str(x)
    get_tablename(URL)
    x = x + 1

In [4]:
#保存项目总表
data_list1 = pd.DataFrame(data_list)
data_list1[['项目名称','所属批次','公示时间','申报楼栋','总层数']] = data_list1[['1项目名称','2所属批次','3公示时间','4申报楼栋','5总层数']]
data_list1 = data_list1.iloc[:,5:]
data_list1.to_excel('C:\\Users\\l\\Desktop\\项目总表.xlsx',encoding = "gbk",index = False)

In [6]:
#获取明细信息
alldata_list = []
def get_tabledata(ID,NO):
    params = {
                'id': ID, #参数一
                'page': '1',
                'size': '10000',
                'sort': '',
                'buildNo': NO,#参数二
                }
    url = 'http://117.39.29.75:8085/pricePublic/house/public/price?' + urlencode(params)
    #print(url)
    response = requests.get(url, headers=headers)
    pat = '<td>(.*?)</td>'
    pat1 = '<span style="display:inline-block;">(.*?)</span></td>'
    pat2 = '<span>(.*?)</span>'
    rst = re.compile(pat).findall(response.text)
    rst1 = re.compile(pat1).findall(response.text)
    rst2 = re.compile(pat2).findall(response.text)
    str = rst2[0]
    result = re.search('(.*?)号楼 一房一价信息,层数：(\d+)', str)
    lz = result.group(1)
    lc = result.group(2)
    
    del rst[0]

    for y in range(0,len(rst),5):
        alldata_list.append({
            '项目名称':rst1[0],
            '所属批次':sort_list[z],
            '公示时间':time_list[z],
            '项目地址':rst1[2],
            '装修标准':rst1[1],
            '楼幢信息':lz,
            '楼层信息':lc,
            '房间号': rst[y+1],
            '建筑面积': rst[y+2],
            '房屋均价': rst[y+3],
            '房屋总价': rst[y+4]
        })

In [7]:
#获取前200个项目明细数据
z = 0
while z < 200:
    ID = name_list[z]
    str = buildNo_list[z]
    m = str.replace("、",",")
    m = m.replace("#","")
    m = m.split(',')
    for NO in m:
        get_tabledata(ID,NO)
    z = z + 1

In [8]:
#保存前200个项目明细数据
alldata_list = pd.DataFrame(alldata_list)
alldata_list.to_excel('C:\\Users\\l\\Desktop\\项目明细200.xlsx',encoding = "gbk",index = False)

In [ ]:
#获取200-399个项目明细数据
z = 200
while z < 400:
    ID = name_list[z]
    str = buildNo_list[z]
    m = str.replace("、",",")
    m = m.replace("#","")
    m = m.split(',')
    for NO in m:
        get_tabledata(ID,NO)
    z = z + 1

In [ ]:
#保存200-399个项目明细数据
alldata_list = pd.DataFrame(alldata_list)
alldata_list.to_excel('C:\\Users\\l\\Desktop\\项目明细400.xlsx',encoding = "gbk",index = False)

In [ ]:
#获取400-615个项目明细数据
z = 400
while z < 615:
    ID = name_list[z]
    str = buildNo_list[z]
    m = str.replace("、",",")
    m = m.replace("#","")
    m = m.split(',')
    for NO in m:
        get_tabledata(ID,NO)
    z = z + 1

In [ ]:
#保存400-615个项目明细数据
alldata_list = pd.DataFrame(alldata_list)
alldata_list.to_excel('C:\\Users\\l\\Desktop\\项目明细616.xlsx',encoding = "gbk",index = False)